In [12]:
import numpy as np
import cv2
from Undistort import Undistort
from Threshold import Threshold
from Warp import Warp
from Polyfit import Polyfit
from Polydraw import Polydraw
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from moviepy.video.io.VideoFileClip import VideoFileClip
from IPython.display import HTML

print('Loaded Modules!!')

Loaded Modules!!


In [13]:
undistort = Undistort()
polydraw = Polydraw()
polyfit = Polyfit()

In [14]:
src = np.float32([
    [580, 460],
    [700, 460],
    [1040, 680],
    [260, 680],
])

dst = np.float32([
    [260, 0],
    [1040, 0],
    [1040, 720],
    [260, 720],
])

In [15]:
warp = Warp(src, dst)
undistort.calibratie_camera()

Working on Image :  ./camera_cal/calibration10.jpg
Working on Image :  ./camera_cal/calibration11.jpg
Working on Image :  ./camera_cal/calibration12.jpg
Working on Image :  ./camera_cal/calibration13.jpg
Working on Image :  ./camera_cal/calibration14.jpg
Working on Image :  ./camera_cal/calibration15.jpg
Working on Image :  ./camera_cal/calibration16.jpg
Working on Image :  ./camera_cal/calibration17.jpg
Working on Image :  ./camera_cal/calibration18.jpg
Working on Image :  ./camera_cal/calibration19.jpg
Working on Image :  ./camera_cal/calibration2.jpg
Working on Image :  ./camera_cal/calibration20.jpg
Working on Image :  ./camera_cal/calibration3.jpg
Working on Image :  ./camera_cal/calibration6.jpg
Working on Image :  ./camera_cal/calibration7.jpg
Working on Image :  ./camera_cal/calibration8.jpg
Working on Image :  ./camera_cal/calibration9.jpg


In [20]:
def process_images_test(base_image, name):

    img_undist = undistort.undistort(base_image)
    mpimg.imsave('./output_images/undistorted_{}'.format(name), img_undist)

    threshold = Threshold(kernel=3)
    img = threshold.combined_threshold(img_undist)
    mpimg.imsave('./output_images/thresholded_{}'.format(name), img, cmap="gray")

    img = warp.warp(img)
    mpimg.imsave('./output_images/warped_{}'.format(name), img, cmap="gray")

    left_fit, right_fit = polyfit.polyfit(img, visualize=True)

    img = polydraw.draw(img_undist, left_fit, right_fit, warp.Minv)

    mpimg.imsave('./output_images/final_{}'.format(name), img, cmap="gray")
    # Measure curvature
    lane_curve, car_position = polyfit.curvature(img)
    font = cv2.FONT_HERSHEY_SIMPLEX

    if car_position > 0:
        cv2.putText(img, 'Radius of curvature (Left)  = %.2f m' % (lane_curve), (10, 40), font, 1,
                    (255, 255, 255), 2, cv2.LINE_AA)
        pos_text = '{}m right of center'.format(car_position)
    else:
        cv2.putText(img, 'Radius of curvature (Right) = %.2f m' % (lane_curve), (10, 70), font, 1,
                    (255, 255, 255), 2, cv2.LINE_AA)
        pos_text = '{}m left of center'.format(abs(car_position))

    cv2.putText(img, "Lane curve: {}m".format(lane_curve.round()), (10, 40), font, 1,
                color=(255, 255, 255), thickness=2)
    cv2.putText(img, "Car is {}".format(pos_text), (10, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, color=(255, 255, 255),
                thickness=2)
    mpimg.imsave('./output_images/annotated_{}'.format(name), img)

In [21]:
test_images = ['straight_lines1.jpg', 'straight_lines2.jpg', 'test1.jpg', 'test2.jpg', 'test3.jpg', 'test4.jpg', 'test5.jpg', 'test6.jpg']
for image in test_images:
    img = mpimg.imread('./test_images/' + image)
    process_images_test(img, image)

557.495984314 m 536.68352728 m
Ratio:  1.03877975749
596.16572661 m 791.288550084 m
Ratio:  0.753411289151
498.103027934 m 505.60591059 m
Ratio:  0.985160611261
551.543685641 m 478.06926176 m
Ratio:  1.15368991432
616.776783178 m 544.637159495 m
Ratio:  1.13245446519
597.038946632 m 564.737733321 m
Ratio:  1.05719683918
536.742908887 m 597.780562606 m
Ratio:  0.897892876522
598.081384725 m 526.610227895 m
Ratio:  1.13571927214


In [22]:
def process_image(base_image):
    img_undist = undistort.undistort(base_image)
    threshold = Threshold(kernel=3)
    img = threshold.combined_threshold(img_undist)
    img = warp.warp(img)
    left_fit, right_fit = polyfit.polyfit(img, visualize=False)
    img = polydraw.draw(img_undist, left_fit, right_fit, warp.Minv)
    # Measure curvature
    lane_curve, car_position = polyfit.curvature(img)
    font = cv2.FONT_HERSHEY_SIMPLEX

    if car_position > 0:
        cv2.putText(img, 'Radius of curvature (Left)  = %.2f m' % (lane_curve), (10, 40), font, 1,
                    (255, 255, 255), 2, cv2.LINE_AA)
        pos_text = '{}m right of center'.format(car_position)
    else:
        cv2.putText(img, 'Radius of curvature (Right) = %.2f m' % (lane_curve), (10, 70), font, 1,
                    (255, 255, 255), 2, cv2.LINE_AA)
        pos_text = '{}m left of center'.format(abs(car_position))

    cv2.putText(img, "Lane curve: {}m".format(lane_curve.round()), (10, 40), font, 1,
                color=(255, 255, 255), thickness=2)
    
    cv2.putText(img, "Car is {}".format(pos_text), (10, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, color=(255, 255, 255),
                thickness=2)
    return img

In [10]:
white_output = 'project_video_done_2.mp4'
clip1 = VideoFileClip('project_video.mp4')
white_clip = clip1.fl_image(process_image)  # NOTE: this function expects color images!!
white_clip.write_videofile(white_output, audio=False)

564.765747273 m 576.495319239 m
Ratio:  0.979653656197
[MoviePy] >>>> Building video project_video_done_2.mp4
[MoviePy] Writing video project_video_done_2.mp4


  0%|          | 1/1261 [00:00<04:55,  4.27it/s]

560.054270287 m 521.183854398 m
Ratio:  1.07458100546


  0%|          | 2/1261 [00:00<04:51,  4.32it/s]

501.70041395 m 451.72254132 m
Ratio:  1.11063842969


...

100%|█████████▉| 1259/1261 [05:37<00:00,  3.79it/s]

598.987584973 m 595.62802543 m
Ratio:  1.00564036513


100%|█████████▉| 1260/1261 [05:37<00:00,  3.86it/s]

612.619291071 m 520.325613186 m
Ratio:  1.17737677244


[MoviePy] Done.
[MoviePy] >>>> Video ready: project_video_done_2.mp4 



In [11]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

In [14]:
challenge = 'challenge_video_done_2.mp4'
clip1 = VideoFileClip('challenge_video.mp4')
white_clip = clip1.fl_image(process_image)  # NOTE: this function expects color images!!
white_clip.write_videofile(challenge, audio=False)

539.579290235 m 446.36363818 m
Ratio:  1.20883343553
[MoviePy] >>>> Building video challenge_video_done_2.mp4
[MoviePy] Writing video challenge_video_done_2.mp4


  0%|          | 1/485 [00:00<01:53,  4.26it/s]

508.890358941 m 532.790143618 m
Ratio:  0.955142216944


  0%|          | 2/485 [00:00<01:52,  4.28it/s]

600.198670136 m 648.092947802 m
Ratio:  0.926099677788


...

100%|█████████▉| 484/485 [01:55<00:00,  4.33it/s]

561.503129801 m 579.949420183 m
Ratio:  0.968193277309


100%|██████████| 485/485 [01:55<00:00,  4.32it/s]

477.617000421 m 512.300779717 m
Ratio:  0.932298015797


[MoviePy] Done.
[MoviePy] >>>> Video ready: challenge_video_done_2.mp4 



In [15]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(challenge))

In [12]:
harder_output = 'harder_challenge_video_done_2.mp4'
clip1 = VideoFileClip('harder_challenge_video.mp4')
white_clip = clip1.fl_image(process_image)  # NOTE: this function expects color images!!
white_clip.write_videofile(harder_output, audio=False)

527.933882247 m 521.638439959 m
Ratio:  1.0120685935
[MoviePy] >>>> Building video harder_challenge_video_done_2.mp4
[MoviePy] Writing video harder_challenge_video_done_2.mp4


  0%|          | 1/1200 [00:00<04:43,  4.23it/s]

532.701477164 m 548.438865723 m
Ratio:  0.971305117958


  0%|          | 2/1200 [00:00<04:43,  4.22it/s]

495.774370672 m 569.991122871 m
Ratio:  0.869793143751


...

100%|█████████▉| 1198/1200 [05:07<00:00,  3.40it/s]

475.638782021 m 523.832725609 m
Ratio:  0.907997455616


100%|█████████▉| 1199/1200 [05:08<00:00,  3.38it/s]

477.852621055 m 492.211745716 m
Ratio:  0.970827342529


[MoviePy] Done.
[MoviePy] >>>> Video ready: harder_challenge_video_done_2.mp4 



In [13]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(harder_output))